In [ ]:
import json
import boto3
import os

s3 = boto3.client('s3')

def lambda_handler(event, context):
    # Get the bucket and key from the S3 event trigger
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key']

    # Read raw JSON file
    raw_obj = s3.get_object(Bucket=bucket, Key=key)
    raw_data = json.loads(raw_obj['Body'].read().decode('utf-8'))

    tracks, albums, artists = [], [], []

    for item in raw_data:
        # Add track
        tracks.append({
            "track_id": item['track_id'],
            "track_name": item['track_name'],
            "popularity": item['popularity'],
            "album_id": item['album_id'],
            "artist_id": item['artist_id']
        })

        # Add album
        albums.append({
            "album_id": item['album_id'],
            "album_name": item['album_name'],
            "release_date": item['release_date']
        })

        # Add artist
        artists.append({
            "artist_id": item['artist_id'],
            "artist_name": item['artist_name']
        })

    # Write to transformed_data/
    s3.put_object(Bucket=bucket, Key='transformed_data/tracks.json', Body=json.dumps(tracks, indent=2))
    s3.put_object(Bucket=bucket, Key='transformed_data/albums.json', Body=json.dumps(albums, indent=2))
    s3.put_object(Bucket=bucket, Key='transformed_data/artists.json', Body=json.dumps(artists, indent=2))

    return {
        'statusCode': 200,
        'body': '✅ Transformation complete and uploaded to transformed_data/'
    }
